# Assignment 3 Question 1

### CO 487/687 Applied Cryptography Fall 2023 

This Jupyter notebook contains Python 3 code for Assignment 3 Question 1 on "Symemtric Encryption in Python".

### Documentation

- [Python cryptography library](https://cryptography.io/en/latest/)

The following code imports all the required functions for the assignment.

In [32]:
import base64
import getpass
import json
import os
import sys
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes    
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import constant_time

These two functions convert a byte array into a printable string and back, which might be helpful to you since cryptographic routines often work with byte arrays.

In [33]:
def bytes2string(b):
    return base64.urlsafe_b64encode(b).decode('utf-8')

def string2bytes(s):
    return base64.urlsafe_b64decode(s.encode('utf-8'))

Implement the main encryption function below. Your function will take as input a string, and will output a string or dictionary containing all the values needed to decrypt (other than the password, of course). The code below will prompt the user to enter their password during encryption.

In [34]:
def encrypt(message):
    
    # encode the string as a byte string, since cryptographic functions usually work on bytes
    plaintext = message.encode('utf-8')

    # Use getpass to prompt the user for a password
    password = getpass.getpass("Enter password:")
    password2 = getpass.getpass("Enter password again:")

    # Do a quick check to make sure that the password is the same!
    if password != password2:
        sys.stderr.write("Passwords did not match")
        sys.exit()

    ### START: This is what you have to change
    
    # Encode the password as a byte string
    password = password.encode('utf-8')
    # Dictionary to save each value (non-private) to be sent 
    cipher_dict = {}
    
    # Salts should be randomly generated
    salt = os.urandom(16)
    # Store value of salt
    cipher_dict["salt"] = salt
    # Get key for encrypting using PBKDF2
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(),
        length=16,
        salt=salt,
        iterations=200000,
    )
    # Length of 128 bits
    key = kdf.derive(password)
    
    # Nonce should be randomly generated, size same as block (128 bits = 16 bytes)
    nonce = os.urandom(16)
    # Store value of nonce
    cipher_dict["nonce"] = nonce 
    # Using AES-128 inn CTR mode to encrypt
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))
    encryptor = cipher.encryptor()
    ct = encryptor.update(plaintext) + encryptor.finalize()
    # Store value of ciphertext
    cipher_dict["ct"] = ct 
    
    # Salt to use in PBKDF2 to obtain key to use in HMAC
    salt_hmac = os.urandom(16)
    # Store value of new salt
    cipher_dict["salt_hmac"] = salt_hmac
    # Get key for HMAC using PBKDF2
    kdf_hmac = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(),
        length=32,
        salt=salt_hmac,
        iterations=200000,
    )
    # Length of 256 bits = 32 bytes, same as used hash output length (recommended)
    key_hmac = kdf_hmac.derive(password)
    h = hmac.HMAC(key_hmac, hashes.SHA3_256())
    h.update(ct)
    # Tag of our ciphertext (Using encrypt-then-MAC approach)
    signature = h.finalize()
    # Store value of tag
    cipher_dict["signature"] = signature
    
    """
    # Use this code instead if you want to return a string with all values rather than a dict
    # Note that decrypt function would need to be slightly adjusted for this change
    salt_str = bytes2string(salt)
    salt_hmac_str = bytes2string(salt_hmac)
    nonce_str = bytes2string(nonce)
    ct_str = bytes2string(ct)
    signature_str = bytes2string(signature)

    return salt_str + salt_hmac_str + nonce_str + ct_str + signature_str
    """
    
    # Return all (non-private) values needed to decrypt 
    # Values are in byte string format. In case of wanting str, would need to convert each value using bytes2string() 
    return cipher_dict
    
    ### END: This is what you have to change

Now we call the `encrypt` function with a message, and print out the ciphertext it generates.

In [35]:
mymessage = "Hello, world!"
ciphertext = encrypt(mymessage)
print(ciphertext)

Enter password: ········
Enter password again: ········


{'salt': b'\x12\xc3\xbc\xa9\xf6\x1a,\x03\xdbB^zDF)\x8d', 'nonce': b'\xf5\xfd\xe2"\xfed\xd1\xcc\xa7\x05WD\xb8g[u', 'ct': b'P\x18\xeb\xb0\xf1\xc5I\x96\x8d#(O\x8d', 'salt_hmac': b'#K\xea\xe8\n\xa0\xbe\x01\xa3\x0fm\xeb\t\xd0\x8a\x18', 'signature': b'\xdb\xa2\xceG\xd2V\xed\xc6\x03+<\x94\xca3\x95=\xbct\x9b\xdfI\xe5\x14\xb7\x03n\x8e\xd1<\x07\xce\x89'}


Implement the main decryption function below.  Your function will take as input the string or dictionary output by `encrypt`, prompt the user to enter the password, and then do all the relevant cryptographic operations.

In [36]:
def decrypt(ciphertext):
    
    # prompt the user for the password
    password = getpass.getpass("Enter the password:")

    ### START: This is what you have to change
    
    # Encode the password as a byte string
    password = password.encode('utf-8')
    
    # Get the salt used to get key for HMAC
    salt_hmac = ciphertext["salt_hmac"]
    # Get same key used for HMAC using PBKDF2 with same values as before
    kdf_hmac = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(),
        length=32,
        salt=salt_hmac,
        iterations=200000,
    )
    key_hmac = kdf_hmac.derive(password)
    
    h = hmac.HMAC(key_hmac, hashes.SHA3_256())
    h.update(ciphertext["ct"])
    h_copy = h.copy() # get a copy of `h' to be reused
    # Verify tag. Important doing this step before decrypting.
    h.verify(ciphertext["signature"])
    
    # Get the salt used to get key for encrypting
    salt = ciphertext["salt"]
    # Get key used for encrypting using PBKDF2
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(),
        length=16,
        salt=salt,
        iterations=200000,
    )
    key = kdf.derive(password)
    
    # Get the nonce used for encrypting
    nonce = ciphertext["nonce"]
    # Using same algorithm as for encrypt, AES-128 on CTR mode
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce))
    decryptor = cipher.decryptor()
    # Decrypt ciphertext
    plaintext = decryptor.update(ciphertext["ct"]) + decryptor.finalize()
    
    ### END: This is what you have to change

    # decode the byte string back to a string
    return plaintext.decode('utf-8')

Now let's try decrypting the ciphertext you encrypted above by entering the same password as you used for encryption.

In [37]:
mymessagedecrypted = decrypt(ciphertext)
print(mymessagedecrypted)
assert mymessagedecrypted == mymessage

Enter the password: ········


Hello, world!


Try again but this time see what happens if you use a different password to decrypt. Your function should fail.

In [38]:
mymessagedecrypted = decrypt(ciphertext)
print(mymessagedecrypted)
assert mymessagedecrypted == mymessage

Enter the password: ········


InvalidSignature: Signature did not match digest.